In [1]:
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
import os 


os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_groq import ChatGroq 
from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [3]:
backend_code = """
\n'I'll help you create a Node.js backend for an e-commerce application similar to Flipkart. We'll use Express.js for the API server and SQLite for the database since it doesn't require a separate server.\n\n<boltArtifact id=\"ecommerce-backend\" title=\"E-commerce Backend Application\">\n<boltAction type=\"file\" filePath=\"package.json\">\n{\n  \"name\": \"ecommerce-backend\",\n  \"version\": \"1.0.0\",\n  \"type\": \"module\",\n  \"scripts\": {\n    \"start\": \"node src/index.js\",\n    \"dev\": \"node --watch src/index.js\"\n  },\n  \"dependencies\": {\n    \"express\": \"^4.18.2\",\n    \"better-sqlite3\": \"^8.5.0\",\n    \"bcryptjs\": \"^2.4.3\",\n    \"jsonwebtoken\": \"^9.0.1\",\n    \"cors\": \"^2.8.5\",\n    \"express-validator\": \"^7.0.1\"\n  }\n}\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/index.js\">\nimport express from 'express';\nimport cors from 'cors';\nimport { router as authRoutes } from './routes/auth.js';\nimport { router as productRoutes } from './routes/products.js';\nimport { router as cartRoutes } from './routes/cart.js';\nimport { router as orderRoutes } from './routes/orders.js';\nimport { initDb } from './db/init.js';\n\nconst app = express();\nconst PORT = 3000;\n\napp.use(cors());\napp.use(express.json());\n\n// Initialize database\ninitDb();\n\n// Routes\napp.use('/api/auth', authRoutes);\napp.use('/api/products', productRoutes);\napp.use('/api/cart', cartRoutes);\napp.use('/api/orders', orderRoutes);\n\napp.listen(PORT, () => {\n  console.log(`Server running on port ${PORT}`);\n});\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/db/init.js\">\nimport Database from 'better-sqlite3';\n\nconst db = new Database('ecommerce.db');\n\nexport function initDb() {\n  // Users table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS users (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      email TEXT UNIQUE NOT NULL,\n      password TEXT NOT NULL,\n      name TEXT NOT NULL,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n    )\n  `);\n\n  // Products table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS products (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      name TEXT NOT NULL,\n      description TEXT,\n      price DECIMAL(10,2) NOT NULL,\n      stock INTEGER NOT NULL,\n      category TEXT NOT NULL,\n      image_url TEXT,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n    )\n  `);\n\n  // Cart table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS cart_items (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      user_id INTEGER NOT NULL,\n      product_id INTEGER NOT NULL,\n      quantity INTEGER NOT NULL,\n      FOREIGN KEY (user_id) REFERENCES users (id),\n      FOREIGN KEY (product_id) REFERENCES products (id)\n    )\n  `);\n\n  // Orders table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS orders (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      user_id INTEGER NOT NULL,\n      total_amount DECIMAL(10,2) NOT NULL,\n      status TEXT NOT NULL,\n      shipping_address TEXT NOT NULL,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n      FOREIGN KEY (user_id) REFERENCES users (id)\n    )\n  `);\n\n  // Order items table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS order_items (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      order_id INTEGER NOT NULL,\n      product_id INTEGER NOT NULL,\n      quantity INTEGER NOT NULL,\n      price DECIMAL(10,2) NOT NULL,\n      FOREIGN KEY (order_id) REFERENCES orders (id),\n      FOREIGN KEY (product_id) REFERENCES products (id)\n    )\n  `);\n}\n\nexport default db;\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/middleware/auth.js\">\nimport jwt from 'jsonwebtoken';\n\nconst JWT_SECRET = 'your-secret-key';\n\nexport function authenticateToken(req, res, next) {\n  const authHeader = req.headers['authorization'];\n  const token = authHeader && authHeader.split(' ')[1];\n\n  if (!token) {\n    return res.status(401).json({ error: 'Authentication required' });\n  }\n\n  jwt.verify(token, JWT_SECRET, (err, user) => {\n    if (err) {\n      return res.status(403).json({ error: 'Invalid token' });\n    }\n    req.user = user;\n    next();\n  });\n}\n\nexport function generateToken(user) {\n  return jwt.sign({ id: user.id, email: user.email }, JWT_SECRET, { expiresIn: '24h' });\n}\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/auth.js\">\nimport express from 'express';\nimport bcrypt from 'bcryptjs';\nimport { body, validationResult } from 'express-validator';\nimport db from '../db/init.js';\nimport { generateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.post('/register',\n  [\n    body('email').isEmail(),\n    body('password').isLength({ min: 6 }),\n    body('name').notEmpty()\n  ],\n  async (req, res) => {\n    const errors = validationResult(req);\n    if (!errors.isEmpty()) {\n      return res.status(400).json({ errors: errors.array() });\n    }\n\n    const { email, password, name } = req.body;\n\n    try {\n      const hashedPassword = await bcrypt.hash(password, 10);\n      const stmt = db.prepare('INSERT INTO users (email, password, name) VALUES (?, ?, ?)');\n      const result = stmt.run(email, hashedPassword, name);\n      \n      const user = { id: result.lastInsertRowid, email, name };\n      const token = generateToken(user);\n      \n      res.status(201).json({ user, token });\n    } catch (error) {\n      res.status(500).json({ error: 'Error creating user' });\n    }\n  }\n);\n\nrouter.post('/login',\n  [\n    body('email').isEmail(),\n    body('password').notEmpty()\n  ],\n  async (req, res) => {\n    const { email, password } = req.body;\n\n    try {\n      const stmt = db.prepare('SELECT * FROM users WHERE email = ?');\n      const user = stmt.get(email);\n\n      if (!user || !(await bcrypt.compare(password, user.password))) {\n        return res.status(401).json({ error: 'Invalid credentials' });\n      }\n\n      const token = generateToken(user);\n      res.json({ user: { id: user.id, email: user.email, name: user.name }, token });\n    } catch (error) {\n      res.status(500).json({ error: 'Error during login' });\n    }\n  }\n);\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/products.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\n// Get all products\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare('SELECT * FROM products');\n  const products = stmt.all();\n  res.json(products);\n});\n\n// Get product by ID\nrouter.get('/:id', (req, res) => {\n  const stmt = db.prepare('SELECT * FROM products WHERE id = ?');\n  const product = stmt.get(req.params.id);\n  \n  if (!product) {\n    return res.status(404).json({ error: 'Product not found' });\n  }\n  \n  res.json(product);\n});\n\n// Add new product (admin only)\nrouter.post('/',\n  authenticateToken,\n  [\n    body('name').notEmpty(),\n    body('price').isFloat({ min: 0 }),\n    body('stock').isInt({ min: 0 }),\n    body('category').notEmpty()\n  ],\n  (req, res) => {\n    const { name, description, price, stock, category, image_url } = req.body;\n    \n    const stmt = db.prepare(`\n      INSERT INTO products (name, description, price, stock, category, image_url)\n      VALUES (?, ?, ?, ?, ?, ?)\n    `);\n    \n    const result = stmt.run(name, description, price, stock, category, image_url);\n    res.status(201).json({ id: result.lastInsertRowid, ...req.body });\n  }\n);\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/cart.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.use(authenticateToken);\n\n// Get cart items\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare(`\n    SELECT ci.*, p.name, p.price, p.image_url\n    FROM cart_items ci\n    JOIN products p ON ci.product_id = p.id\n    WHERE ci.user_id = ?\n  `);\n  \n  const cartItems = stmt.all(req.user.id);\n  res.json(cartItems);\n});\n\n// Add to cart\nrouter.post('/',\n  [\n    body('product_id').isInt(),\n    body('quantity').isInt({ min: 1 })\n  ],\n  (req, res) => {\n    const { product_id, quantity } = req.body;\n    \n    const stmt = db.prepare(`\n      INSERT INTO cart_items (user_id, product_id, quantity)\n      VALUES (?, ?, ?)\n    `);\n    \n    const result = stmt.run(req.user.id, product_id, quantity);\n    res.status(201).json({ id: result.lastInsertRowid, ...req.body });\n  }\n);\n\n// Update cart item quantity\nrouter.put('/:id',\n  body('quantity').isInt({ min: 1 }),\n  (req, res) => {\n    const stmt = db.prepare(`\n      UPDATE cart_items\n      SET quantity = ?\n      WHERE id = ? AND user_id = ?\n    `);\n    \n    stmt.run(req.body.quantity, req.params.id, req.user.id);\n    res.json({ message: 'Cart updated' });\n  }\n);\n\n// Remove from cart\nrouter.delete('/:id', (req, res) => {\n  const stmt = db.prepare('DELETE FROM cart_items WHERE id = ? AND user_id = ?');\n  stmt.run(req.params.id, req.user.id);\n  res.json({ message: 'Item removed from cart' });\n});\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/orders.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.use(authenticateToken);\n\n// Get user orders\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare(`\n    SELECT o.*, oi.product_id, oi.quantity, oi.price, p.name\n    FROM orders o\n    JOIN order_items oi ON o.id = oi.order_id\n    JOIN products p ON oi.product_id = p.id\n    WHERE o.user_id = ?\n  `);\n  \n  const orders = stmt.all(req.user.id);\n  res.json(orders);\n});\n\n// Create new order\nrouter.post('/',\n  [\n    body('shipping_address').notEmpty()\n  ],\n  (req, res) => {\n    const { shipping_address } = req.body;\n    \n    db.transaction(() => {\n      // Get cart items\n      const cartItems = db.prepare(`\n        SELECT ci.*, p.price\n        FROM cart_items ci\n        JOIN products p ON ci.product_id = p.id\n        WHERE ci.user_id = ?\n      `).all(req.user.id);\n      \n      if (cartItems.length === 0) {\n        return res.status(400).json({ error: 'Cart is empty' });\n      }\n      \n      // Calculate total\n      const totalAmount = cartItems.reduce((sum, item) => sum + (item.price * item.quantity), 0);\n      \n      // Create order\n      const orderStmt = db.prepare(`\n        INSERT INTO orders (user_id, total_amount, status, shipping_address)\n        VALUES (?, ?, ?, ?)\n      `);\n      \n      const orderResult = orderStmt.run(req.user.id, totalAmount, 'PENDING', shipping_address);\n      const orderId = orderResult.lastInsertRowid;\n      \n      // Create order items\n      const itemStmt = db.prepare(`\n        INSERT INTO order_items (order_id, product_id, quantity, price)\n        VALUES (?, ?, ?, ?)\n      `);\n      \n      cartItems.forEach(item => {\n        itemStmt.run(orderId, item.product_id, item.quantity, item.price);\n      });\n      \n      // Clear cart\n      db.prepare('DELETE FROM cart_items WHERE user_id = ?').run(req.user.id);\n      \n      res.status(201).json({\n        orderId,\n        totalAmount,\n        status: 'PENDING',\n        items: cartItems\n      });\n    })();\n  }\n);\n</boltAction>\n\n<boltAction type=\"shell\">\nnpm install && npm run dev\n</boltAction>\n\nThis sets up a complete e-commerce backend with:\n\n1. User authentication (register/login)\n2. Product management\n3. Shopping cart functionality\n4. Order processing\n5. SQLite database\n\nThe API endpoints are:\n\nAuthentication:\n- POST /api/auth/register\n- POST /api/auth/login\n\nProducts:\n- GET /api/products\n- GET /api/products/:id\n- POST /api/products\n\nCart:\n- GET /api/cart\n- POST /api/cart\n- PUT /api/cart/:id\n- DELETE /api/cart/:id\n\nOrders:\n- GET /api/orders\n- POST /api/orders\n\nYou can test the API endpoints using tools like Postman or curl.'\n\n"
"""

In [4]:
from typing import TypedDict, List, Literal
class TestCase(TypedDict):
    test_id : str 
    description : str 
    steps : List[str]
    status : Literal["draft", "pass", "fail"] = "draft"

In [5]:
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser 
json_output_parser = JsonOutputParser()

json_prompt_template = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : json_output_parser.get_format_instructions()}
)

In [6]:
test_cases_system_prompt = """
You are an elite backend QA engineer and test automation architect, specialized in building high-coverage test suites for mission-critical systems. Your task is to read backend code (provided in a markdown file format), analyze the logic, and generate robust, comprehensive test cases in a structured JSON format.

### Objective
Given a markdown file containing backend code (in Python, Node.js, Java, etc.), your goal is to:
- **Understand** the code logic, functionality, and input/output contracts.
- **Generate well-structured test cases** covering positive, negative, and edge scenarios.
- **Ensure high code coverage** within each module, function, or class.
- **Generate 2 to 5 test cases** per given code block to ensure comprehensive testing.

### Chain of Thought

1. **Understand**:
    - Extract and interpret backend code blocks from the markdown file.
    - Identify all function definitions, input parameters, return types, and conditions.

2. **Basics**:
    - Determine the functional intent of each unit (function/class/method).
    - Identify logical branches, preconditions, and exception handling.

3. **Break Down**:
    - Separate the code into individual testable units.
    - Identify distinct execution paths and their potential outcomes.

4. **Analyze**:
    - Identify normal (expected inputs) and edge/invalid (unexpected/null/empty/wrong type) scenarios.
    - Assess side effects or system integrations (e.g., DB calls, HTTP endpoints).

5. **Build**:
    - Generate test cases in a structured JSON format:
        - **Test Case ID**: A unique identifier for each test case.
        - **Description**: A brief explanation of what the test case does.
        - **Steps**: A list of steps to execute the test case.
        - **Status**: The status of the test case, which should always be set to `"draft"`.
    - **Include coverage** for:
        - Successful execution.
        - Failure conditions.
        - Error handling.
        - Boundary conditions.

6. **Edge Cases**:
    - Include test cases for:
        - Empty strings.
        - Null inputs.
        - Extreme values.
        - Invalid types.

7. **Final Answer**:
    - Output test cases in a JSON format, using the following structure:
    ```json
    [
      {
        "test_id": "unique_test_id_001",
        "description": "Brief description of what the test case does",
        "steps": [
          "Step 1: Description of the first step",
          "Step 2: Description of the second step",
          ...
        ],
        "status": "draft"
      },
      {
        "test_id": "unique_test_id_002",
        "description": "Description of the second test case",
        "steps": [
          "Step 1: Another step",
          "Step 2: Further step",
          ...
        ],
        "status": "draft"
      }
    ]
    ```
    - The **status** of all test cases should always be `"draft"`.
    - If necessary, mock external dependencies (e.g., database calls, HTTP requests) as needed for unit testing.

8. **Tips**:
    - **Cover both expected and unexpected cases** (normal, edge, and failure).
    - **Do not assume any system dependencies are working** — mock them if needed.
    - **Clearly define assertions or expected outputs** for each test case.
    - **Do not guess functionality** — your test cases should be based strictly on the code logic.

### What Not to Do:
- **Never** ignore edge or boundary cases.
- **Do not** paraphrase the code — test cases must reflect true logic from the source.
- **Never** guess the functionality without evidence from the code.
- **Avoid** generating test cases that lack assertions or clear expected outputs.
- **Never** generate non-deterministic or flaky test cases.
- **Do not** assume external dependencies work — mock them if needed.
"""

In [7]:
def generate_test_cases_from_llm(backend_code):
    user_query =  f"Create test cases for the this project backend code: {backend_code}" 
    chain = json_prompt_template | llm  | json_output_parser
    response = chain.invoke({"system_prompt" : test_cases_system_prompt, "human_query" : user_query})
    return response

In [8]:
test_cases = generate_test_cases_from_llm(backend_code)
test_cases

[{'test_id': 'auth_register_001',
  'description': 'Register a new user with valid credentials.',
  'steps': ['Step 1: Send a POST request to /api/auth/register with a valid email, password (min 6 characters), and name.',
   'Step 2: Verify that the response status code is 201.',
   'Step 3: Verify that the response body contains a user object with id, email, and name.',
   'Step 4: Verify that the response body contains a token.'],
  'status': 'draft'},
 {'test_id': 'auth_register_002',
  'description': 'Attempt to register a new user with an invalid email format.',
  'steps': ['Step 1: Send a POST request to /api/auth/register with an invalid email, a valid password, and a valid name.',
   'Step 2: Verify that the response status code is 400.',
   'Step 3: Verify that the response body contains an errors array indicating the email is invalid.'],
  'status': 'draft'},
 {'test_id': 'auth_register_003',
  'description': 'Attempt to register a new user with a password shorter than 6 char

In [9]:
revised_test_cases_system_prompt = """
You are a highly skilled QA engineer, specializing in revising and optimizing backend test cases. Your task is to review and revise the existing test cases (provided in the 'old_test_cases' format) based on the following considerations:
- **User Feedback**: The feedback provided by the end users regarding the existing test cases (provided in 'user_feedback').
- **Backend Code**: The updated backend code file (provided in a markdown format).
- **Test Case Improvement**: Enhance the existing test cases for better coverage, clarity, and accuracy, ensuring alignment with the new or updated code and any feedback.

### Objective:
- **Revise** the existing test cases to better reflect the functionality of the updated backend code.
- **Incorporate feedback** from users regarding existing test cases, improving the coverage of edge cases, clarity, and accuracy.
- **Ensure high code coverage** across the entire backend system, including positive, negative, edge, and failure scenarios.
- **Keep test cases structured** in a clean and well-organized format.
  
### Chain of Thought:

1. **Understand**:
    - Analyze the updated backend code provided in markdown format.
    - Review the existing test cases (from 'old_test_cases') to understand their current structure and coverage.
    - Consider the feedback from the user (provided in 'user_feedback') to identify any gaps, confusion, or areas for improvement in the existing tests.

2. **Align**:
    - Ensure that the test cases accurately cover the functionality, input/output contracts, and edge conditions as described in the backend code.
    - Make sure that the test cases now reflect any changes in logic, new features, or bug fixes from the backend code.
    - Cross-check the revised test cases with the user feedback to ensure all reported issues or misunderstandings are addressed.

3. **Refine**:
    - Revise the test cases to include:
        - Clearer descriptions of what each test is verifying.
        - Well-defined steps to reproduce the test case.
        - Expected outcomes and assertions.
    - Remove any redundant test cases or those that no longer make sense due to changes in the backend code.
    - Ensure that edge cases, null inputs, and extreme values are properly tested.

4. **Improve**:
    - Enhance the coverage for:
        - Normal flow.
        - Failure scenarios (e.g., invalid input, exceptions).
        - Boundary conditions (e.g., empty, null, maximum input).
    - Ensure that no test case is left unaddressed based on user feedback.
    - If necessary, create new test cases to cover missed paths in the backend logic.

5. **Generate**:
    - Revise and return the updated test cases in a structured format, using the following template:

    json
    [
      {
        "test_id": "unique_test_id_001",
        "description": "Revised description of the test case",
        "steps": [
          "Step 1: Description of the first step",
          "Step 2: Description of the second step",
          ...
        ],
        "expected_result": "Expected outcome or assertion for the test case",
        "status": "draft"
      }
    ]

    - Keep the **status** of all test cases as "draft".
    - If necessary, mock external dependencies (e.g., databases, external APIs) to isolate backend logic in unit tests.

6. **Final Answer**:
    - Your output should be the revised test cases in JSON format, following the structure above. Ensure that the test cases reflect the updated backend logic, are comprehensive, and address any user feedback provided.

### Key Guidelines:
- Ensure that test cases are **consistent with the latest backend code** and **incorporate user feedback** effectively.
- **Revise existing test cases**, improving their clarity, coverage, and correctness.
- Address any gaps or errors in the current test cases as highlighted by user feedback.
- **Keep the tests focused on business logic**, with proper validation for all expected and edge cases.
- **Do not guess the functionality**; base the revisions on code analysis and user feedback.

### What Not to Do:
- **Never leave old or outdated test cases unchanged** if they no longer match the updated backend code.
- **Do not ignore user feedback**—ensure it is reflected in the revised test cases.
- **Avoid revising tests without clarity on backend code behavior**—always base revisions on actual code logic and user feedback.
"""

In [10]:
def revised_test_cases_from_llm(test_cases, backend_code, user_feedback):
    user_query =  f"Revise this test cases {test_cases} for this backend code {backend_code} with this user feedback {user_feedback}" 
    chain = json_prompt_template | llm  | json_output_parser
    response = chain.invoke({"system_prompt" : revised_test_cases_system_prompt, "human_query" : user_query})
    return response


In [11]:
user_feedback = "The test cases are not covering all the scenarios"
revised_test_cases = revised_test_cases_from_llm(test_cases, backend_code, user_feedback)

In [12]:
revised_test_cases

[{'test_id': 'auth_register_001',
  'description': 'Register a new user with valid credentials.',
  'steps': ['Step 1: Send a POST request to /api/auth/register with a valid email, password (min 6 characters), and name.',
   'Step 2: Verify that the response status code is 201.',
   'Step 3: Verify that the response body contains a user object with id, email, and name.',
   'Step 4: Verify that the response body contains a token.'],
  'expected_result': 'Response status code should be 201. Response body should contain a user object and a token.',
  'status': 'draft'},
 {'test_id': 'auth_register_002',
  'description': 'Attempt to register a new user with an invalid email format.',
  'steps': ["Step 1: Send a POST request to /api/auth/register with an invalid email (e.g., 'invalid-email'), a valid password, and a valid name.",
   'Step 2: Verify that the response status code is 400.',
   'Step 3: Verify that the response body contains an errors array indicating the email is invalid.'],
